In [85]:
import sys, os
sys.path.append('/home/A00512318/TCN')
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim
import torch.nn.functional as F
from TCN.mnist_pixel.utils import data_generator
from TCN.mnist_pixel.model import TCN
import numpy as np
import argparse

# set default values for model
batch_size = 64
cuda = True
dropout = 0.15803481115567097
clip = -1
epochs = 10
kernel_size = 6
levels = 8
log_interval = 100
lr = 0.0002099003405580982
optimm = 'Adam'
nhid = 26
seed = 1111
permutee = False
root = '../data/fashion_mnist'
save_filename = './checkpoints/fashionmnist/fashionmnist_real_lookbest_3.pt'
input_channels = 1
n_classes = 10
seq_length = int(784 / input_channels)
steps = 0

In [81]:
from torchvision import datasets, transforms

def data_generator(root, batch_size):
    train_set = datasets.FashionMNIST(root=root, train=True, download=True,
                               transform=transforms.Compose([
                                   transforms.ToTensor(),
                                   transforms.Normalize((0.1307,), (0.3081,))
                               ]))
    test_set = datasets.FashionMNIST(root=root, train=False, download=True,
                              transform=transforms.Compose([
                                  transforms.ToTensor(),
                                  transforms.Normalize((0.1307,), (0.3081,))
                              ]))

    train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size)
    test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size)
    return train_loader, test_loader

train_loader, test_loader = data_generator(root, batch_size)

classes = ('T-shirt/Top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot')
# print(classes)

In [82]:
def trainTCN(ep):
    global steps
    train_loss = 0
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        if cuda: 
            data, target = data.to(device), target.to(device)
        data = data.view(-1, input_channels, seq_length)
        if permutee:
            data = data[:, :, permute]
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        if clip > 0:
            torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()
        train_loss += loss.item()
        steps += seq_length
        if batch_idx > 0 and batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}\tSteps: {}'.format(
                ep, batch_idx * batch_size, len(train_loader.dataset),
                100. * batch_idx / len(train_loader), train_loss/log_interval, steps))
            train_losses_[ep-1].append((train_loss/log_interval, steps))
            train_loss = 0  

In [83]:
def testTCN():
    test_loss = 0
    correct = 0
    correct_class = list(0. for i in range(10))
    correct_total = list(0. for i in range(10))
    tot = 0
    confusion_matrix = [[] for i in range(len(classes))]
    with torch.no_grad():
        for data, target in test_loader:
            if cuda:
                data, target = data.to(device), target.to(device)
            data = data.view(-1, input_channels, seq_length)
            if permutee:
                data = data[:, :, permute]
            output = model(data)
            test_loss += F.nll_loss(output, target, size_average=False).item()
            _, pred = torch.max(output, 1)
            correct += pred.eq(target.data.view_as(pred)).cpu().sum()
#             print(len(target.data.view_as(pred)))
            c = (pred == target).squeeze()
            tot += 1
#             if tot != 313:
#             for i in range(len(test_loader.dataset) // batch_size):
# #                     print(pred[i], target.data.view_as(pred)[i])
#                 print(i)
#                 label = pred[i]
#                 if (pred[i] == target.data.view_as(pred)[i]):
#                     correct_class[label] += c[i].item()
#                 correct_total[label] += 1
                    
                
#     print(tot)
#     for i in range(10):
#         print('Accuracy of %5s : %2d %%' % (
#             classes[i], 100 * correct_class[i] / correct_total[i]))

    test_loss /= len(test_loader.dataset)
#     print(correct.item())
    accuracies_.append(correct.item() / 10000.)
    
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    return test_loss

In [86]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

permute = torch.Tensor(np.random.permutation(784).astype(np.float64)).long()
channel_sizes = [nhid] * levels
model = TCN(input_channels, n_classes, channel_sizes, kernel_size=kernel_size, dropout=dropout)

model = nn.DataParallel(model)
model.to(device)

# permute = permute.cuda()
permute = permute.to(device)
optimizer = getattr(optim, optimm)(model.parameters(), lr=lr)
accuracies_ = []
train_losses_ = [[] for i in range(0, epochs)]
scores = []
for epoch in range(1, epochs+1):
    trainTCN(epoch)
    testTCN()
    if epoch % 5 == 0: 
        print('Saving checkpoint for model.....')
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.module.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'train_losses': train_losses_,
            'accuracies': accuracies_,
            'curr_lr': lr,
        }, save_filename)
    if epoch % 10 == 0:
        lr /= 10
        for param_group in optimizer.param_groups:
            param_group['lr'] = lr
    torch.save(model.module.state_dict(), save_filename)
    scores.append({
        'accuracies': accuracies_,
        'train_losses': train_losses_,
    })
    print('Saved as %s' % save_filename) 

Train Epoch: 1 [6400/60000 (11%)]	Loss: 2.333956	Steps: 79184
Train Epoch: 1 [12800/60000 (21%)]	Loss: 1.611906	Steps: 157584
Train Epoch: 1 [19200/60000 (32%)]	Loss: 1.121247	Steps: 235984
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.921388	Steps: 314384
Train Epoch: 1 [32000/60000 (53%)]	Loss: 0.817358	Steps: 392784
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.765209	Steps: 471184
Train Epoch: 1 [44800/60000 (75%)]	Loss: 0.724044	Steps: 549584
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.699181	Steps: 627984
Train Epoch: 1 [57600/60000 (96%)]	Loss: 0.668623	Steps: 706384

Test set: Average loss: 0.6760, Accuracy: 7591/10000 (75%)

Saved as ./checkpoints/fashionmnist/fashionmnist_real_lookbest_3.pt
Train Epoch: 2 [6400/60000 (11%)]	Loss: 0.649692	Steps: 814576
Train Epoch: 2 [12800/60000 (21%)]	Loss: 0.619453	Steps: 892976
Train Epoch: 2 [19200/60000 (32%)]	Loss: 0.629884	Steps: 971376
Train Epoch: 2 [25600/60000 (43%)]	Loss: 0.604885	Steps: 1049776
Train Epoch: 2 [32000/60000 (53%)]	Loss: 

In [72]:
loaded_model = None
with open(save_filename, 'rb') as f:
        loaded_model = torch.load(f)
#         print(model)
        
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
count_parameters(model)

36400

In [ ]:
loaded_model.to(device)
# kernel = loaded_model.module.tcn.network[0].net[0].weight.data.clone()
# kernel.shape
# kernel
# loaded_model.module.tcn.network
loaded_model.module.tcn.network[0].net

In [ ]:
fig = plt.figure()
plt.figure(figsize=(10,10))
for idx, filt  in enumerate(kernel):
#     print(filt[0, :])
#     print(filt[0, :])
#     plt.subplot(4,7, idx + 1)
    plt.imshow(filt, cmap="gray")
    plt.axis('off')
    
    
fig.show()

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np

# Data for plotting
t = np.arange(1, 21)
# s = 1 + np.sin(2 * np.pi * t)

fig, ax = plt.subplots()
ax.plot(t, accuracy_)

ax.set(xlabel='Epoch', ylabel='Accuracy',
       title='TCN')
ax.grid()

fig.savefig("test.png")
plt.show()

In [ ]:
mode

In [ ]:
model.to